#Molecular Simulation Using **Robosample**

In this lab we will simulate SARS-COV-2 Mpro protease with Robosample using its Python interface. Then, we will run a short rigid body dynamics HMC simulation.

<h1>Theoretical Aspects</h1>

Molecules conformations lie in complex highly dimensional spaces, and present a challange for molecular simulations that try to explore them. Take, for example, ethane: although it is a simple molecule, it already has $8 \times 3 = 24$ dimensions. To simplify this endevour some types of degrees of freedom may be constrained: such as bond lengths. This will increase the integration time step and reduce the number equations solved at each step.





<figure>
<center>
<img src='https://github.com/spirilaurentiu/Notebooks/blob/main/ethane.s.png?raw=true' />
<figcaption>FIGURE 1. Ethane Molecule </figcaption>
</center>
</figure>



In the case of biological macromolecules, their particular chemistry may induce certain patterns that can be speculated for the use of constraints. For example, proteins have secondary structures elements such as helices, that can be treated as rigid bodies during some parts of the simulation. 




<figure>
<center>
<img src='https://github.com/spirilaurentiu/Notebooks/blob/main/protein.s.png?raw=true' />
<figcaption>FIGURE 2. Proteins secondary structures </figcaption>
</center>
</figure>

Using constraints, by definition, do not take into account all the degrees of freedom therefore suffers from a lack of ergodicity. Also, particular degrees of freedom may be coupled when transitioning from a conformational basin to another. We overcome this in Robosample through the means of "overlaped blocked Gibbs sampling" described below.

Gibbs sampling is a Markov Chain Monte Carlo algorithm which samples from a multivariate distribution $\pi(X = \{x_i\})$ by taking turns in sampling each of its component $x_i$, conditioned on the previous sampled components $\pi(x_i | \{x_{0,i-1}\})$.

In a similar fashion, the algorithm allows sampling jointly multiple components $x_{i, i+k}$ in what is called "blocked Gibbs sampling". Even more so, the components may overlap to form "overlapped blocked Gibbs sampling".

We will use this particular type of Gibbs sampling in Robosample by alternating multiple definitions of rigid bodies.
 Robosample is a molecular simulation program that uses algorithms designed primarly for robot mechanics. 




# Part 0 - Download and Install Robosample

---



Robosample binaries are already build on Google colab and uploaded on Github. Clone the Github repository that contains Robosample binaries.

In [14]:
%cd /content/
! if [ -d Notebooks ]; then echo "Notebooks exists"; else git clone --recursive https://github.com/spirilaurentiu/Notebooks.git; fi
!ls Notebooks
%cd Notebooks/
!git pull

/content
Notebooks exists
ala10	      inp.test	     README.md	       simulate.py  tools
build	      mproc	     robosample.ipynb  temp
ethane.s.png  protein.s.png  robots	       test.ipynb
/content/Notebooks
Already up to date.


 Now we have to make sure that Robosample binary finds its libraries. Set the linker path.
 Robosample relies heavly on GPUs. Set cuda directories.

In [3]:
!if echo $LD_LIBRARY_PATH | grep Notebook; then echo "LD_LIBRARY_PATH set";else echo "Run one of the below commands"; fi
%env LD_LIBRARY_PATH=/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/content/Notebooks/build/release/robosample/src
!echo "$LD_LIBRARY_PATH"

!if echo $CUDA_ROOT | grep usr; then echo "CUDA directories set"; else echo "Run one of the below commands"; fi
%env CUDA_INC_DIR=/usr/local/cuda
%env CUDA_ROOT=/usr/local/cuda


/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/content/Notebooks/build/release/robosample/src
LD_LIBRARY_PATH set
env: LD_LIBRARY_PATH=/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/content/Notebooks/build/release/robosample/src
/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/content/Notebooks/build/release/robosample/src
/usr/local/cuda
CUDA directories set
env: CUDA_INC_DIR=/usr/local/cuda
env: CUDA_ROOT=/usr/local/cuda


If everything went okay so far, Robosample should run on a test small molecule.

In [4]:
%cd build/release/robosample/src/
!./GMOLMODEL_robo inp.2but

/content/Notebooks/build/release/robosample/src
Reading input...
Dumping input file:
  BOOST_MDSTEPS : 1 
  BOOST_TEMPERATURE : 600 
  DIHEDRAL : 0 1 2 3 
  DISTANCE : 0 1 
  FFSCALE : AMBER 
  FIXMAN_POTENTIAL : TRUE 
  FIXMAN_TORQUE : TRUE 
  FLEXFILE : ligand.td.flex 
  GBSA : 1.0 
  GEOMETRY : FALSE 
  INPCRD : ligand.inpcrd 
  MDSTEPS : 10 
  MOLECULES : 2but 
  NMA_OPTION : 0 
  NONBONDED_CUTOFF : 2 
  NONBONDED_METHOD : 1 
  OPENMM : TRUE 
  OPENMM_CalcOnlyNonbonded : TRUE 
  OUTPUT_DIR : temp 
  PRINT_FREQ : 1 
  PRMTOP : ligand.prmtop 
  RANDOM_WORLD_ORDER : FALSE 
  RBFILE : ligand.rb 
  REPRODUCIBLE : FALSE 
  ROOTS : 0 
  ROOT_MOBILITY : Free 
  ROUNDS : 10 
  ROUNDS_TILL_REBLOCK : 10 
  RUN_TYPE : Normal 
  SAMPLER : VV 
  SAMPLES_PER_ROUND : 1 
  SEED : 42 
  TEMPERATURE_FIN : 300 
  TEMPERATURE_INI : 300 
  THERMOSTAT : Andersen 
  THREADS : 0 
  TIMESTEPS : 0.003 
  VISUAL : FALSE 
  WORLDS : R1 
  WRITEPDBS : 0 
Done.
Molecule directory: 2but
tcmalloc: large alloc 1073

Now we need to install Robosample python interface

In [5]:
# MDTraj is a prerequisite for robosample.py
!if pip list | grep mdtraj ; then echo "MDTraj is installed"; else pip install mdtraj; fi

# Robosample Python interface is in the tools directory
%env PYTHONPATH=/env/python:/content/Notebooks/tools
!echo $PYTHONPATH

%cd /content/Notebooks/

     |████████████████████████████████| 2.1 MB 8.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mdtraj: filename=mdtraj-1.9.7-cp37-cp37m-linux_x86_64.whl size=5533639 sha256=1f4eb75cd0f2f9a458fc19a193ad5e6fb6f6bc134dc995697add60827f6a91e9
  Stored in directory: /root/.cache/pip/wheels/6f/84/9d/6854e5264a4423209de26a6e4b34d13750289c72ba0439bbfd
Successfully built mdtraj
env: PYTHONPATH=/env/python:/content/Notebooks/tools
/env/python:/content/Notebooks/tools
/content/Notebooks


If everything went OK we should be able to set up a simple example with the Python

In [6]:
!ls

ala10  ethane.s.png  protein.s.png  robosample.ipynb  test.ipynb
build  mproc	     README.md	    simulate.py       tools


In [ ]:
# Run a test script
! if [ -d temp ]; then echo "temp directory exists"; else mkdir -p temp/pdbs; fi
!ls
!python simulate.py

In [8]:
!ls temp/pdbs

final.bot01.0000000005.pdb    sb.bot01.0.s0.0000000003.pdb
sb.bot01.0.s0.0000000000.pdb


# Part 1 - Prepare a Robosample Script

---

<h1>Why use a script?<!h1>

Molecular dynamics simulations require a lot of information about
* Input data
  * coordinates
  * topology
  * which atoms are included in energy terms
  * parameters for functions in energy terms
* System description
  * periodicity
  * constraints
* Integrators
  * algorithms to propagate forward in time
  * adjust box size
  * adjust kinetic energy (temperature)
* Simulation
  * how long to run
  * how much output data to store

  Let's go through what the parameters mean and I'll give you values for a simple simulation of MPro C-terminal part.



![picture]("https://cdn.rcsb.org/images/structures/li/2liz/2liz_models.jpeg")

In [60]:
# Get molecule
!ls mproc/

2liz.pdb  ligand.min.rst7  ligand.prmtop


<h1>Import necessary modules</h1>
First we need to import the module that contains the Robosample interface, robosample.py. Also import here oher modules that we may need.

In [ ]:
import sys
sys.path.append("/content/Notebooks/tools/")

In [28]:
#!touch /content/__init__.py
#!touch /content/Notebooks/__init__.py
#!touch /content/Notebooks/tools/__init__.py

# Imports
import os
from robosample import *


<h1>Load molecular structure files</h1>


In [33]:
# Load Amber files
prmtop = AmberPrmtopFile("mproc/ligand.prmtop")
inpcrd = AmberInpcrdFile("mproc/ligand.min.rst7")

# Hardware platform
platform = Platform.getPlatformByName('GPU')
properties={'nofThreads': 0}


GPU


<h1>Create System <!>

In [34]:
# Remove any existing robots directory
if os.path.exists("robots"):
  shutil.rmtree("robots")

# Create a Robosample system by calling createSystem on prmtop
system = prmtop.createSystem(createDirs = True,
	nonbondedMethod = "CutoffPeriodic",
 	nonbondedCutoff = 1.44*nanometer,
 	constraints = None,
 	rigidWater = True,
 	implicitSolvent = True,
 	soluteDielectric = 1.0,
 	solventDielectric = 78.5,
 	removeCMMotion = False
)


<h1> Choose an Integrator <!h1>

In [35]:
# Choose an integrator
integrator = HMCIntegrator(300*kelvin,   # Temperature of head bath
                           0.001*picoseconds) # Time step

<h1>Wrap everything in a Simulation Object</h1>

In [36]:
# Create Simulation object
simulation = Simulation(prmtop.topology, system, integrator, platform, properties)
simulation.reporters.append(PDBReporter('temp', 3))
simulation.context.setPositions(inpcrd.positions)

Starting Simulation init
Robosample directory set to /content/Notebooks
Robosample executable set to /content/Notebooks/build/release/robosample/src/GMOLMODEL_robo
DEBUG ['bot.prmtop']
Done Simulation init
robots/bot0/bot.rst7 ['robots/bot0/bot.prmtop']
Calculating distance matrix...
Calculating dssp...
coils [[  0  13]
 [ 28  40]
 [ 50  57]
 [ 71  74]
 [ 88 108]
 [111 120]]
flatClusts [1 7 4 4 7 1 1 4 7 5 2 1 3 5 1 3 7 6 2 4 2 3 4 1 7 6 6 7 7 5 7 4 4 1 4 6 4
 7 1 4 1 7 2 1 6 7 2 2 7 5 1 2 7 7 7 2 7 7 4 2 1 7 7 4 2 1 3 2 2 2 7 4 3 5
 1 7 6 2 6 7 7 4 7 4 1 7 1 2 3 4 4 6 1 2 7 6 1 3 3 2 2 6 4 3 4 6 7 4 1 4 7
 1 7 3 2 5 4 1 2 7 6 7 2 3 2 5 2 6 2 2 1 6 3 1 2 1 4 2 2 1 4 4 5 7 1 4 6 3
 4 1 2 7 6 1 6 5 7 3 2 3 3 7 5 2 2 2 2 3 2 4 1 3 4 4 3 2 2 2 3 1 3 1 4 2 2
 7 7 7 2 7 6 2 2 3 6 1 2 4 4 4 3 6 1 2 2 5 7 1 4 1 6 3 3 1 7 1 3 7 1 4 1 4
 4 2 1 4 7 6 7 2 1 6 4 4 4 5 7 7 7 7]
0 [4, 12, 'CA', 0, 'C', 0, array([ 1.0200005,  0.1549997, -1.150999 ])] [4, 12, 'CA', 0, 'C', 0, array([ 1.0200005,  0.1549

<h1> Add Worlds </h1>

In [37]:
## Generate NMA-Scaled Flex Files
sAll = [[1, 120]]

simulation.addWorld(regionType='stretch',
                    region=sAll,
                    rootMobility='Weld',
                    timestep=0.004,
                    mdsteps=5,
                    argJointType='Pin',
                    subsets=['rama','side'],
                    samples=1)


Starting Simulation addWorld
region [[  1 120]]
flatRegion [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120]
Wrote to bot.stretch.2.flex
Done Simulation addWorld


<h1>Run the simulation </h1>

In [38]:

# Run simulation
simulation.step(5)



Starting Simulation step
Done Simulation step


In [32]:
!ls temp/pdbs/

final.bot01.0000000005.pdb    sb.bot01.0.s0.0000000003.pdb
sb.bot01.0.s0.0000000000.pdb


**QUESTION** 